<a href="https://colab.research.google.com/github/anmaxwell/UniNotebooks/blob/master/KTrainWorking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 177kB/s 
     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 6.7MB 35.4MB/s 
     |████████████████████████████████| 983kB 42.4MB/s 
     |████████████████████████████████| 245kB 54.1MB/s 
     |████████████████████████████████| 778kB 46.6MB/s 
     |████████████████████████████████| 471kB 52.3MB/s 
     |████████████████████████████████| 276kB 55.6MB/s 
     |████████████████████████████████| 450kB 47.4MB/s 
     |████████████████████████████████| 3.9MB 49.8MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 1.1MB 52.6MB/s 
     |████████████████████████████████| 890kB 53.7MB/s 
     |████████████████████████████████| 3.0MB 49.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.19.3-cp36-none-any.whl size=25267699 sha256=da206884b9d080a0c0949764cfdfd29505e8583c4e6a53e8f1a421b9d49724ff
  Stored in directory: /root/.cache/pip/wheels/f5/0a/d6/edae48b

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [3]:
data_train = pd.read_csv('/content/Train.csv')

data_train.shape

(3179, 2)

In [4]:
data_train.head()

,comment,issue
0,install sent otdl,0
1,new install sent OTDL,0
2,Sent OTD Needed PYE update,0
3,Update OTD v26.01\nCust installed it but want ...,0
4,OTD Link purchased Sage 50Cloud Payroll\nprovi...,0


In [5]:

data_test = pd.read_csv('/content/Test.csv')
data_test.head()

,comment,issue
0,23423) ODBC Microsoft Access Driver Login Fail...,0
1,#Corona sent asa's 46376 & 46392,0
2,fps crash otd 26.1 sent,1
3,FPS Issues - Crashing (Quick Fix) FPS Crashing...,1
4,installer doing nothing auto updater doing not...,1


In [6]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df = data_train,
                   text_column = 'comment',
                   label_columns=['issue'],
                   val_df = data_test,
                   maxlen = 128,
                   preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [7]:
X_train[0].shape

(3179, 128)

In [8]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc=preprocess)

Is Multi-Label? False
maxlen is 128
done.


In [9]:
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 32)

In [10]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 3179 samples, validate on 700 samples
3179/3179 [==============================] - 4177s 1s/sample - loss: 0.4042 - accuracy: 0.8065 - val_loss: 0.3351 - val_accuracy: 0.8400


In [11]:
learner.evaluate()

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       514
           1       0.71      0.67      0.69       186

    accuracy                           0.84       700
   macro avg       0.80      0.78      0.79       700
weighted avg       0.84      0.84      0.84       700



array([[464,  50],
       [ 62, 124]])

In [12]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [13]:
data3 = ['It keeps crashing']

In [14]:
predictor.predict(data3)

['0']

In [15]:
data4 = ['where is the latest download']

In [16]:
predictor.predict(data4)

['0']

In [17]:
data5 = ['there seems to be an issue']

In [18]:
predictor.predict(data5)

['0']

In [19]:
# Saving Json file
model_json = learner.model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [20]:
# Saving Weights
learner.model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
# Save the model for later use
predictor.save("./callcommentbert")

In [22]:
myPredictor = ktrain.load_predictor('./callcommentbert')

In [23]:
myPredictor.predict('this is a known issue')

'0'

In [24]:
import csv

f = open('/content/Reader11May.csv', 'rb')
#csv_f = csv.reader(f)

for num,row in enumerate(csv.reader(f)):
#for row in csv_f:
  print(myPredictor.predict(row),row)

FileNotFoundError: ignored